# Task: Implementing the Lesk Algorithm and Evaluating its Accuracy

## Description
In this task, you are required to implement the Lesk algorithm from scratch, avoiding the use of any existing implementations such as nltk.
## Steps

1. **Data Preparation**
   - Obtain the SemCor corpus, which is annotated with WordNet synsets. You can download it from the following URL: [SemCor Corpus](http://web.eecs.umich.edu/~mihalcea/downloads.html).

2. **Implementation and Evaluation**
   - Extract 50 sentences from the SemCor corpus.
   - For each sentence, perform word sense disambiguation for at least one noun. Use the Lesk algorithm that you have implemented.
   - Calculate the accuracy of your implemented system by comparing the assigned senses with the annotated senses in SemCor.

3. **Randomization and Average Accuracy**
   - Randomly select 50 sentences from the SemCor corpus.
   - Randomly select nouns from these sentences for word sense disambiguation.
   - Run the word sense disambiguation process multiple times (e.g., 10 times).
   - Calculate the average accuracy of your system's performance across these executions.

4. **corpus-Lesk Algorithm**
   - Implement the Lesk algorithm using the SemCor corpus.




In [10]:
import nltk
import random
from nltk.corpus import wordnet as wn
from nltk.corpus import semcor
from nltk.corpus.reader.wordnet import Lemma
from nltk import word_tokenize
import os
import numpy as np
import string

nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('semcor')

[nltk_data] Downloading package wordnet to /home/palius/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/palius/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package semcor to /home/palius/nltk_data...
[nltk_data]   Package semcor is already up-to-date!


True


## Overview
This code implements a simplified version of the Lesk algorithm for word sense disambiguation using WordNet synsets. The Lesk algorithm aims to determine the most suitable sense of a word within a given context by comparing the overlap of the word's signature with the context.

## Functions

### `remove_punctuation(sentence)`
This function removes punctuation from a sentence, excluding hyphens and underscores.

### `signature(sense, stop_words)`
This function computes the signature of a WordNet synset. The signature includes the synset's definition and words from its examples, excluding specified stop words.

### `compute_overlap(signature, context)`
This function calculates the overlap between a synset's signature and a given context in terms of the number of shared words.

### `extract_nouns_index(sentence, pos)`
This function identifies the indexes of nouns in a sentence based on their POS tags.

### `simplified_lesk(word, sentence, stop_words)`
This function performs a simplified Lesk algorithm. It finds the synset that best matches the given word within a specific sentence context, using the overlap between synset signatures and context words.

### `semcor_extraction(sentence_number=50)`
This function extracts a specified number of sentences containing random nouns from the SemCor corpus. It returns the extracted sentences along with the extracted noun lemmas.

### `select_lemma(lemmas)`
This function randomly selects a lemma from a list of lemmas, ensuring it represents a single word.

### `remove_word(sentence, word)`
This function removes a chosen word from a sentence.

## Workflow - utility functions
1. The code imports necessary libraries and modules.
2. Various utility functions are defined for text processing and Lesk algorithm steps.
3. The path to the SemCor corpus is provided.
4. Stop words are loaded from a file.
5. The `semcor_extraction()` function is defined to extract random sentences and their associated noun lemmas from SemCor.
6. The `select_lemma()` function is provided to select a lemma that represents a single word.
7. The `remove_word()` function removes a chosen word from a sentence.




In [2]:
def remove_punctuation(sentence):
    """
    Remove punctuation from a sentence.

    Args:
        sentence: The sentence from which to remove punctuation.

    Returns:
        The sentence without punctuation.
    """
    for character in string.punctuation:
        if character not in ['-', '_']:
            sentence = sentence.replace(character, '')
    return sentence


def signature(sense, stop_words):
    """
    Compute the signature of a WordNet synset, namely its name
    and the words contained in its examples.

    Args:
        sense: The WordNet synset.
        stop_words: A list of words to ignore.

    Returns:
        The set of words that compose the signature of `sense`.
    """
    s = sense.definition()
    for e in sense.examples():
        s = s + " " + e
    s = remove_punctuation(s)
    s = {i for i in s.split() if i not in stop_words}
    return s


def compute_overlap(signature, context):
    """
    Compute the size of the overlap between two sets of words.

    Args:
        signature: The signature set.
        context: The context set.

    Returns:
        The size of the overlap between the two sets.
    """
    return len(signature & context)


def extract_nouns_index(sentence, pos):
    """
    Find the indexes of all the nouns in the sentence.

    Args:
        sentence: The input sentence.
        pos: The part-of-speech tags of the words in the sentence.

    Returns:
        A list of indexes indicating the positions of nouns in the sentence.
    """
    return [i for i, p in enumerate(pos) if p == 'NN']


def simplified_lesk(word, sentence, stop_words):
    """
    Perform a simplified version of Lesk's algorithm to assign
    the most likely WordNet synset to a word.

    Args:
        word: The word for which to find the synset.
        sentence: The sentence in which the word appears for disambiguation.
        stop_words: A list of words to ignore.

    Returns:
        The most likely synset for the word.
    """
    synsets = wn.synsets(word)
    if not synsets:
        return None

    best_sense = synsets[0]
    max_overlap = 0
    context = set(sentence)

    for sense in synsets:
        sign = signature(sense, stop_words)
        overlap = compute_overlap(sign, context)
        if overlap > max_overlap:
            max_overlap = overlap
            best_sense = sense

    return best_sense


# Load stop words
with open("stop_words_FULL.txt") as fp:
    stop_words = {l.strip() for l in fp.readlines()}


def semcor_extraction(sentence_number=50):
    """
    Extract a specified number of sentences and their extracted lemmas from SemCor.

    Args:
        sentence_number: The number of sentences to extract.

    Returns:
        A list of extracted sentences and a list of extracted lemmas.
    """
    sentences = []
    extracted = []

    total_sentences = len(semcor.tagged_sents(tag='both'))
    all_tagged_sents = semcor.tagged_sents(tag='both')

    for _ in range(sentence_number):
        # Choose a random index to select a sentence
        random_index = random.randint(0, total_sentences - 1)

        # Extract nouns from the sentence with random_index
        nouns = [sentence_tree for sentence_tree in all_tagged_sents[random_index] if
                 isinstance(sentence_tree.label(), Lemma) and
                 sentence_tree[0].label() == "NN"]

        # Choose a random noun from the nouns list and extract it from the random_index sentence
        if nouns:
            random_noun = random.choice(nouns)
            lemma = random_noun.label()
            extracted.append(lemma)
            sentence = " ".join(semcor.sents()[random_index])
            # sentences.append(remove_word(sentence, lemma.name()))
            sentences.append(sentence)
    return sentences, extracted


def select_lemma(lemmas):
    """
    Select a lemma that has a single word.

    Args:
        lemmas: A list of lemmas.

    Returns:
        The selected lemma.
    """
    right_lemma = False
    while not right_lemma:
        lemma = random.choice(lemmas)
        if "_" not in lemma:
            return lemma


def remove_word(sentence, word):
    """
    Remove the chosen word from the sentence.

    Args:
        sentence: The input sentence.
        word: The word to be removed.

    Returns:
        The sentence without the chosen word.
    """
    tokens = word_tokenize(sentence)
    filtered_sentence = [w for w in tokens if w != word]
    return filtered_sentence


In [3]:
# Number of sentences to extract
num_sentences = 50

# Number of program runs
num_runs = 10
semcor_sentences, extracted_nouns =semcor_extraction(num_sentences)




## Evaluation Process

1. **Accuracies List Initialization**
   - An empty list called `accuracies` is created to store the accuracy of each run.

2. **Loop for Multiple Runs**
   - The code enters a loop that runs `num_runs` times to evaluate accuracy over multiple iterations.

3. **Disambiguation Loop**
   - Inside the loop:
     - Sentences and corresponding nouns are extracted from SemCor using `semcor_extraction`.
     - An empty list named `results` is created to hold disambiguation results.

4. **Disambiguating and Comparing**
   - For each sentence-noun pair:
     - The `simplified_lesk` function is used to perform sense disambiguation on the noun in the sentence.
     - The disambiguated sense is compared with the sense annotated in SemCor for evaluation.

5. **Recording and Calculating Results**
   - If the disambiguated sense matches the SemCor sense, 1 is added to `results`; otherwise, 0 is added.
   - Accuracy for the current run is calculated as the sum of correct disambiguations divided by the total number of pairs.

6. **Accuracies Collection**
   - The calculated accuracy for the current run is added to the `accuracies` list.

7. **Calculating Average Accuracy**
   - After all runs, the code calculates the average accuracy across all runs using the `np.mean` function.

8. **Printing Results**
   - The average accuracy over `num_runs` runs is printed to the console.



In [11]:


# List to store accuracy of each run
accuracies = []

for _ in range(num_runs):
    # Extract sentences and nouns from SemCor
    # sentences, extracted_nouns = semcor_extraction(num_sentences)

    # List to store results of disambiguation
    results = []

    for sentence, noun in zip(semcor_sentences, extracted_nouns):
        # Disambiguate noun in sentence
        sense = simplified_lesk(noun.name(), sentence, stop_words)
        print(f"Phrase: {sentence}")
        print(f"sense from lesk: {sense}")
        print(f"sense from semcor : {noun.synset()}")
        # Check if disambiguated sense matches annotated sense in SemCor
        if sense == noun.synset():
            results.append(1)
        else:
            results.append(0)

    # Calculate accuracy for this run
    accuracy = sum(results) / len(results)
    accuracies.append(accuracy)

# Calculate average accuracy over all runs
average_accuracy = np.mean(accuracies)

print(f"Average accuracy over {num_runs} runs: {average_accuracy}")

Phrase: We of the liberal led world got all set for peace and rehabilitation .
sense from lesk: Synset('rehabilitation.n.01')
sense from semcor : Synset('reclamation.n.01')
Phrase: Howard ( the thick middle-aged man ) was looking at her .
sense from lesk: Synset('man.n.01')
sense from semcor : Synset('man.n.01')
Phrase: But he had in Walter Hendl a willing conductor able only up to a point .
sense from lesk: Synset('point.n.01')
sense from semcor : Synset('degree.n.02')
Phrase: After Captain Docherty sent Arleigh Griffith for Hoag he was able to complete his detailed inspection of the third floor and to receive a report from his man covering the floors above before Griffith returned , buoyed up by a brief stop for another glass of champagne .
sense from lesk: Synset('glass.n.01')
sense from semcor : Synset('glass.n.03')
Phrase: Will the house on any part-time farm you are considering make a satisfactory full-time residence ?
sense from lesk: Synset('farm.n.01')
sense from semcor : Syns

# Performance Consideration

## Evaluation Results
After running the Lesk algorithm for word sense disambiguation using your provided code, here are some key observations based on the output:

- The Lesk algorithm attempted to disambiguate word senses for a set of sentences extracted from the SemCor corpus.
- For each sentence, the algorithm assigned a sense using the `simplified_lesk` function and compared it to the annotated sense from SemCor.
- The output shows instances where the disambiguated sense matched the annotated sense and instances where it did not.

## Accuracy Analysis
- The provided output includes disambiguation results for different sentences, along with the senses assigned by the Lesk algorithm and the annotated senses from SemCor.
- The calculated average accuracy over 10 runs is approximately 0.45 (45%).
- This average accuracy indicates that the Lesk algorithm's performance is moderate, suggesting that it correctly identified the sense of the target words in about 45% of the cases.

## Factors Impacting Performance
Several factors could contribute to the observed accuracy:

- **Sense Overlap**: The algorithm's accuracy heavily depends on the overlap between the context words and the words in the synset signature. If there's limited overlap, the algorithm may struggle to identify the correct sense.
  
- **Context Complexity**: Complex sentence structures or sentences with multiple possible interpretations can challenge the algorithm's accuracy.

- **Quality of Annotations**: The accuracy also depends on the quality of annotations in the SemCor corpus. If the corpus contains ambiguous or incorrect annotations, it can affect the evaluation.

